# Spectral decomposition

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)  
This work by Jephian Lin is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from scipy import linalg as LA
import sympy

In [ ]:
def spec_decom(A, tol=0.0001):                            ### A:symmeetric matrix
    n = A.shape[0]
    vals,vecs = LA.eigh(A)                                ### eigenvalues and corresponding eigenvectors of A

    inds = np.where((vals[1:] - vals[:-1]) > tol)[0]      ### the index that the eigenvalue are different from next 
    starts = np.hstack([np.array([0]), inds+1])           ### the start index of each distinct eigenvalue 
    ends = np.hstack([inds+1, np.array([n])])             ### the end index of each distinct eigenvalue+1
    
    dist_vals = vals[starts]                              ### the set of distinct eigenvalues
    projs = np.zeros((len(dist_vals), n, n))              ### construct (number of distinct eigenvalues) n*n matrix
    i = 0
    for s,e in zip(starts,ends):
        space = vecs[:,s:e]                               ### the matrix whose columns are the eigenvectors for the corresponding eigenvalues
        projs[i] = space.dot(space.T)                     ### the projection matrix for the eigenspace
        i += 1
    
    return dist_vals, projs                               ### return the distinct eigenvalues and each projection matrix

## Main idea

A matrix $A$ is **symmetric** if $A^\top = A$.  

Every symmetric matrix $A$ can be written as  
$$ A = \sum_{i=1}^q \lambda_i P_i$$
such that $\sum_{i=1}^q P_i = I$, $P_iP_j = O$ if $i\neq j$, and $P_i^2 = P_i$.  
This is called the **spectral decomposition** of $A$.  

Each $P_i$ is in fact a projection matrix onto a space $E_i$.  
For any vector ${\bf v}$ in $E_i$, $A{\bf v} = \lambda_i{\bf v}$.  
We call $\lambda_i$ an **eigenvalue** ,  
call ${\bf v}$ an **eigenvector** ,  
and $E_i$ an **eigenspace** of $A$.

### Note
$$ A = \sum_{i=1}^q \lambda_i V_i {V_i}^\top $$ where $V_i$ is the matrix whose columns are the eigenvectors corresponding to $\lambda_i$

## Side stories

- eigenvalue, eigenvector, eigenspace
- spectral decomposition = projection then scale

## Experiments

###### Exercise 1
Let  
```python
A = np.ones((5,5), dtype=float) - np.eye(5, dtype=float)
v1 = np.ones((5,), dtype=float)
v2 = np.array([1,-1,0,0,0], dtype=float)
```

###### 1(a)
Check if ${\bf v}_1$ is an eigenvector of $A$.  
If yes, what is the eigenvalue?

In [ ]:
A = np.ones((5,5), dtype=float) - np.eye(5, dtype=float)
v1 = np.ones((5,), dtype=float)
v2 = np.array([1,-1,0,0,0], dtype=float)
Av1 = np.dot(A,v1)
v1,Av1

In [ ]:
Av1==4*v1

yes, eigenvalue = 4

###### 1(b)
Check if the $i$-th entry of ${\bf v}_2$ is zero then the $i$-th entry of $A{\bf v}_2$ is also zero.  

In [ ]:
Av2 = np.dot(A,v2)
v2,Av2

yes

###### 1(c)
Check if ${\bf v}_2$ is an eigenvector of $A$.  
If yes, what is the eigenvalue?

In [ ]:
Av2==-1*v2

yes, eigenvalue = -1

###### Exercise 2
Let  
```python
A = np.array([[2,-1,-1],
              [-1,2,-1],
              [-1,-1,2]]) / 3
v1 = np.array([1,1,1])
v2 = np.array([1,-1,0])
```
Since checking the zero is tiring, we will use a different approach to test whether a vector is an eigenvector.  
The following are equivalent:  
- $A{\bf v} = \lambda{\bf v}$ for some $\lambda$
- $A{\bf v}$ is a multiple of ${\bf v}$
- $A{\bf v}$ and ${\bf v}$ are parallel (zero vectors are considered to be parallel to any vector)
- $|\langle A{\bf v}, {\bf v}\rangle| = \|A{\bf v}\|\|{\bf v}\|$

###### 2(a)
Compute $|\langle A{\bf v}_1, {\bf v}_1\rangle|$ and $\|A{\bf v}_1\|\|{\bf v}_1\|$.  
Then see if they are close.

In [ ]:
A = np.array([[2,-1,-1],
              [-1,2,-1],
              [-1,-1,2]]) / 3
v1 = np.array([1,1,1])
v2 = np.array([1,-1,0])
Av1 = A.dot(v1)

In [ ]:
dot = np.abs(np.dot(Av1,v1))
norm = np.linalg.norm(Av1)*np.linalg.norm(v1)
dot,norm

yes, they are close

###### 2(b)
Do the same for ${\bf v}_2$.

In [ ]:
Av2 = A.dot(v2)
dot = np.abs(np.dot(Av2,v2))
norm = np.linalg.norm(Av2)*np.linalg.norm(v2)
dot,norm

yes, they are close

###### 2(c)
Run and understand the code below.  

```python
%matplotlib notebook

tol = 0.1
vs = 5*np.random.randn(3,10000)
Avs = A.dot(vs)
dots = np.abs(np.sum(vs * Avs, axis = 0)) ### |<Avi, vi>|'s
Avs_length = np.linalg.norm(Avs, axis=0) ### ||Avi||'s
vs_length = np.linalg.norm(vs, axis=0) ### ||vi||'s
diff = np.abs( dots - Avs_length*vs_length)
mask = (diff < tol) ### mask for potential eigenvectors
eigvecs = vs[:,mask]

ax = plt.axes(projection='3d')
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)
ax.set_zlim(-5,5)
ax.scatter(eigvecs[0], eigvecs[1], eigvecs[2])
```
What is the plane in the output?  
Change `tol` to 1 or 2.  Then you will see a vague straight line appears.  Why?

In [ ]:
%matplotlib notebook

tol = 0.1
vs = 5*np.random.randn(3,10000)
Avs = A.dot(vs)
dots = np.abs(np.sum(vs * Avs, axis = 0)) ### |<Avi, vi>|'s
Avs_length = np.linalg.norm(Avs, axis=0) ### ||Avi||'s
vs_length = np.linalg.norm(vs, axis=0) ### ||vi||'s
diff = np.abs( dots - Avs_length*vs_length)
mask = (diff < tol) ### mask for potential eigenvectors
eigvecs = vs[:,mask]

ax = plt.axes(projection='3d')
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)
ax.set_zlim(-5,5)
ax.scatter(eigvecs[0], eigvecs[1], eigvecs[2])

The plane is the eigenspace corresponding to $\lambda =1$

In [ ]:
%matplotlib notebook

tol = 2
vs = 5*np.random.randn(3,10000)
Avs = A.dot(vs)
dots = np.abs(np.sum(vs * Avs, axis = 0)) ### |<Avi, vi>|'s
Avs_length = np.linalg.norm(Avs, axis=0) ### ||Avi||'s
vs_length = np.linalg.norm(vs, axis=0) ### ||vi||'s
diff = np.abs( dots - Avs_length*vs_length)
mask = (diff < tol) ### mask for potential eigenvectors
eigvecs = vs[:,mask]

ax = plt.axes(projection='3d')
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)
ax.set_zlim(-5,5)
ax.scatter(eigvecs[0], eigvecs[1], eigvecs[2])

The vague straight line is the eigenspace corresponding to $\lambda =0$.

The number of dots are infected by 2 reasons. Firstly, if the multiplicity an eigenvalue is larger, then the dimension of the corresponding eigenspace is also larger; therefore, there will be more dots appear. Secondly, if the eigenvalue is 0, then the dots appear on the eigenspace will be less. The reason is that $A{\bf v}\approx {\bf 0}$ for ${\bf v}$ near the eigenspace, so the angle $\theta$ between $A{\bf v}$ and $\bf v$ may be large. Then $$|\langle A{\bf v},{\bf v}\rangle |-\|A{\bf v}\|\|{\bf v}\|=\|A{\bf v}\|\|{\bf v}\|(1-\cos\theta)$$ may be larger. 

By the 2 reasons above, there are only few dots appear on the line and it need a larger "tol".

## Exercises

###### Exercise 3
Let  
```python
A = np.array([[0,1,0,0,0,1], 
              [1,0,1,0,0,0], 
              [0,1,0,1,0,0], 
              [0,0,1,0,1,0], 
              [0,0,0,1,0,1], 
              [1,0,0,0,1,0]])
vals,projs = spec_decom(A)
```
Here `vals` are all the $\lambda_i$'s and `projs` are all the $P_i$'s.  

###### 3(a)
Check if $A = \sum_{i=0}^3\lambda_iP_i$.  
Can you do it without any for loop?  
Note:  You might need `np.set_printoptions(precision=1, suppress=True)`.

In [ ]:
A = np.array([[0,1,0,0,0,1], 
              [1,0,1,0,0,0], 
              [0,1,0,1,0,0], 
              [0,0,1,0,1,0], 
              [0,0,0,1,0,1], 
              [1,0,0,0,1,0]])
vals,projs = spec_decom(A)
projst = projs.transpose([1,2,0])
B = projst.dot(vals)
np.set_printoptions(precision=1, suppress=True)
print(B)

In [ ]:
np.isclose(A,B)

yes, $A = \sum_{i=0}^3\lambda_iP_i$

###### 3(b)
Let  
```python
v = np.ones((6,))
```
Check if $A{\bf v} = \sum_{i=0}^3 \lambda_i P_i{\bf v}$.

In [ ]:
v = np.ones((6,))
Av = A.dot(v)
B = np.zeros_like(Av)
for i in range(len(vals)):
    lpv = (vals[i]*projs[i]).dot(v)
    B = B+lpv
print(Av,B)

yes, $A{\bf v} = \sum_{i=0}^3 \lambda_i P_i{\bf v}$

###### 3(c)
Pick any $0\leq i,j\leq 3$.  
Check if $P_iP_j = O$.

In [ ]:
for i in range(4):
    for j in range(i+1,4):
        print(np.dot(projs[i],projs[j]))

yes, $P_iP_j = O$ for any $i\neq j$

###### 3(d)
Pick any $0\leq i\leq 3$.  
Check if $P_i^2 = P_i$.

In [ ]:
for i in range(4):
    pi2 = np.dot(projs[i],projs[i])
    print(np.isclose(pi2,projs[i]).all())

yes, $P_i^2 = P_i$ for all $i$

##### Exercise 4
Let  
```python
%matplotlib inline
plt.axis('equal')
A = np.array([[2,1],
              [1,2]])
t = np.linspace(0, 2*np.pi, 36)
xs = np.cos(t)
ys = np.sin(t)
vs = np.vstack([xs, ys])
Avs = A.dot(vs)
```

###### 4(a)
Plot a red point at the origin.  
Plot the points (columns) in `vs` using `c=t` .  
Then plot the points (columns) in `Avs` using `c=t` .  
Interpret the output using the spectral decomposition.

In [ ]:
%matplotlib inline
plt.axis('equal')
A = np.array([[2,1],
              [1,2]])
t = np.linspace(0, 2*np.pi, 36)
xs = np.cos(t)
ys = np.sin(t)
vs = np.vstack([xs, ys])
Avs = A.dot(vs)
plt.scatter(0,0,c='r')
plt.scatter(vs[0],vs[1],c=t)
plt.scatter(Avs[0],Avs[1],c=t)

For each ${\bf v}$ in vs, ${\bf v}$ can be written as $c_1{\bf v}_1+c_2{\bf v}_2$ where ${\bf v}_1 = \begin{bmatrix}1 \\ 1 \end{bmatrix}$ and ${\bf v}_2 = \begin{bmatrix}-1 \\ 1 \end{bmatrix}$, the eigenvectors of $A$. Since $A{\bf v}_1 = 3{\bf v}_1 \text{ and } A{\bf v}_2 = {\bf v}_2$, the direction of ${\bf v}_1$ would be stretched 3 times and the direction of ${\bf v}_2$ would remain the same.

###### 4(b)
Do the same with  
```python
A = np.array([[1,1],
              [1,1]])
```

In [ ]:
plt.axis('equal')
A = np.array([[1,1],
              [1,1]])
t = np.linspace(0, 2*np.pi, 36)
xs = np.cos(t)
ys = np.sin(t)
vs = np.vstack([xs, ys])
Avs = A.dot(vs)
plt.scatter(0,0,c='r')
plt.scatter(vs[0],vs[1],c=t)
plt.scatter(Avs[0],Avs[1],c=t)

Similarly, the eigenvectors are ${\bf v}_1 = \begin{bmatrix}1 \\ 1 \end{bmatrix}$ and ${\bf v}_2 = \begin{bmatrix}-1 \\ 1 \end{bmatrix}$ with $\lambda_1 = 2 \text{ and }\lambda_2 = 0$, so the the direction of ${\bf v}_1$ would be stretch 2 times and the direction of ${\bf v}_2$ would be compressed to 0.

##### Exercise 5
Let  
```python
A = np.array([[0,1,0,0,0,1], 
              [1,0,1,0,0,0], 
              [0,1,0,1,0,0], 
              [0,0,1,0,1,0], 
              [0,0,0,1,0,1], 
              [1,0,0,0,1,0]])
vals,projs = spec_decom(A)

k = 3
```

###### 5(a)
Let  
```python
B = k*A
vals_B,projs_B = spec_decom(B)
```
Find a relation between `vals` and `vals_B` using `k` .  
Find a relation between `projs` and `projs_B` .

In [ ]:
np.set_printoptions(precision=8, suppress=False)
A = np.array([[0,1,0,0,0,1], 
              [1,0,1,0,0,0], 
              [0,1,0,1,0,0], 
              [0,0,1,0,1,0], 
              [0,0,0,1,0,1], 
              [1,0,0,0,1,0]])
vals,projs = spec_decom(A)
k = 3
B = k*A
vals_B,projs_B = spec_decom(B)

In [ ]:
vals,vals_B

vals_B = 3vals

In [ ]:
np.isclose(projs,projs_B)

projs = projs_B

###### 5(b)
Let  
```python
B = A + k*np.eye(6)
vals_B,projs_B = spec_decom(B)
```
Find a relation between `vals` and `vals_B` using `k` .  
Find a relation between `projs` and `projs_B` .

In [ ]:
B = A + k*np.eye(6)
vals_B,projs_B = spec_decom(B)

In [ ]:
vals,vals_B

vals_B = vals+3

In [ ]:
np.isclose(projs,projs_B)

projs = projs_B

###### 5(c)
It looks like $\lambda=1$ is an eigenvalue of $A$.  
Use `sympy.Matrix` to find the null space of $A - \lambda I$.  
Find a projection matrix of it, and compare it with the corresponding projection matrix in `projs` .

In [ ]:
A = sympy.Matrix(A)
A1 = A-sympy.Matrix.eye(A.shape[0])
A1.nullspace()

In [ ]:
v1 = A1.nullspace()[0]
v2 = A1.nullspace()[1]
V = v1.col_insert(1,v2)
VtV_inv = (V.T*V).inv()
proj = V*VtV_inv*(V.T)

In [ ]:
proj

In [ ]:
projs[2]

The two matrices are the same